# fetch-fonts
This notebook tests converting TTF to vector fonts.

Code source: https://gist.github.com/CatherineH/499a312a04582a00e7559ac0c8f133fa

## download font

In [5]:
!wget https://github.com/adobe-fonts/source-sans/raw/release/TTF/SourceSans3-Regular.ttf

--2023-08-12 20:23:47--  https://github.com/adobe-fonts/source-sans/raw/release/TTF/SourceSans3-Regular.ttf
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/adobe-fonts/source-sans/release/TTF/SourceSans3-Regular.ttf [following]
--2023-08-12 20:23:48--  https://raw.githubusercontent.com/adobe-fonts/source-sans/release/TTF/SourceSans3-Regular.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 431196 (421K) [application/octet-stream]
Saving to: ‘SourceSans3-Regular.ttf’

SourceSans3-Regular 100%[===================>] 421.09K  1.06MB/s    in 0.4s    

2023-08-12 20:23:49 (1.06 MB/s) - ‘SourceSans3-

In [6]:
!ls

fetch-fonts.ipynb  SourceSans3-Regular.ttf  Untitled.ipynb


## convert font

In [19]:
import sys

In [ ]:
!sudo apt-get install python3-fontforge

In [21]:
sys.path.append('/usr/lib/python3/dist-packages/')`
import fontforge

In [23]:
def ttf2svg(i, o):
    font = fontforge.open(i)
    font.generate(o)
    font.close()

In [24]:
ttf2svg('SourceSans3-Regular.ttf', 'SourceSans3-Regular.svg')

The following table(s) in the font have been ignored by FontForge
  Ignoring 'DSIG' digital signature table


In [25]:
!ls

fetch-fonts.ipynb	 SourceSans3-Regular.ttf
SourceSans3-Regular.svg  Untitled.ipynb


In [28]:
!head -5 SourceSans3-Regular.svg

<?xml version="1.0" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd" >
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1">
<metadata>
Created by FontForge 20201107 at Tue Apr  4 06:27:06 2023


### wrap the converting, use temp file

In [29]:
import tempfile

In [54]:
import os

In [57]:
from bs4 import BeautifulSoup

In [75]:
def ttf2xml(i: str):
    with tempfile.TemporaryDirectory() as dir:
        font = fontforge.open(i)
        file_address = os.path.join(dir, i + '._generated.svg')
        font.generate(file_address)
        font.close()
        with open(file_address, mode='r') as fp:
            xml_data = fp.read()
            glyphs = BeautifulSoup(xml_data, 'xml').find_all('glyph')

    return glyphs

In [70]:
from collections import namedtuple

In [81]:
Glyph = namedtuple('Glyph', ['d', 'glyph_name', 'unicode'])

In [76]:
xmls = ttf2xml('SourceSans3-Regular.ttf')

The following table(s) in the font have been ignored by FontForge
  Ignoring 'DSIG' digital signature table


In [82]:
def xml2tuple(xml):
    return [
        Glyph(item.get('d'), item.get('glyph_name'), item.get('unicode'))
        for item
        in xml
    ]

In [83]:
glyphs = xml2tuple(xmls)

In [86]:
len(glyphs)

2487